In [ ]:
# NIH data
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)


#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

from config2 import *

from base2 import (width, height, radius, calc_target_coordinates_centered,
                  calc_rad_angle_from_coordinates)

import numpy as np
target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * \
              (np.pi/180)
target_coords = calc_target_coordinates_centered(target_angs)

In [ ]:
from behav_proc import *

In [ ]:
df_all = pd.read_pickle('/home/demitau/data_Quentin/full_experiments/data2/df_all_3.09.23.pkl.zip')

In [ ]:
df_all_9jan = pd.read_pickle('/home/demitau/data_Quentin/full_experiments/data2/df_all2.pkl' )

In [ ]:
len(df_all), len(df_all_9jan)

In [ ]:
df_all.describe() - df_all_9jan.describe()

In [ ]:
tgt_inds_all = list( df_all['target_inds'].unique() )  + [None]

In [ ]:
#from figure.plot_behav2 import getSubDf, getColn, computeErrSensVersions
envs_cur = ['stable', 'random']
block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ (0,1) ]
tgt_inds_cur = tgt_inds_all
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'  # observed - goal, motor performance error

In [ ]:
# compute separately for variable shift sizes to generate grids later
# separately to avoid filtering problems later (I could get too many values per subj 
# due to multiple rows with diffrent trial shift size and same trial)
envs_cur = ['stable', 'random', 'all']
tgt_inds_cur = tgt_inds_all
#%debug
df_all_multi_tsz, ndf2vn = computeErrSensVersions(df_all, envs_cur,block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
       dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
        coln_nh = 'non_hit_not_adj',
        coln_nh_out = 'non_hit_shifted',
        subj_list = subjects, error_type=error_type,
        trial_shift_sizes = [1],
                 addvars=[])

assert not df_all_multi_tsz.duplicated().any() 

In [ ]:
# compute separately for variable shift sizes to generate grids later
# separately to avoid filtering problems later (I could get too many values per subj 
# due to multiple rows with diffrent trial shift size and same trial)
envs_cur = ['stable', 'random', 'all']
tgt_inds_cur = tgt_inds_all
#%debug
computation_ver='computeErrSens3'
df_all_multi_tsz3, ndf2vn = computeErrSensVersions(df_all, envs_cur,block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
       dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
        coln_nh = 'non_hit_not_adj',
        coln_nh_out = 'non_hit_shifted',
        computation_ver=computation_ver,
        subj_list = subjects, error_type=error_type,
        trial_shift_sizes = [1],
                 addvars=[])

computation_ver='computeErrSens2'
df_all_multi_tsz2, ndf2vn = computeErrSensVersions(df_all, envs_cur,block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
       dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
        coln_nh = 'non_hit_not_adj',
        coln_nh_out = 'non_hit_shifted',
        computation_ver=computation_ver,
        subj_list = subjects, error_type=error_type,
        trial_shift_sizes = [1],
                 addvars=[])

assert not df_all_multi_tsz.duplicated().any() 

In [ ]:
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']

In [ ]:
def plot_violins(df):
    coln = 'err_sens'
    #dfc_multi_tsz  = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard')
    dfc0_multi_tsz  = truncateDf(df, coln, q=0.0, infnan_handling='discard')

    qs = 'trial_shift_size == 1 and trial_group_col_calc == "trialwe"'
    qs += ' and non_hit_not_adj'
    #dfc  = dfc_multi_tsz.query(qs)
    dfc0 = dfc0_multi_tsz.query(qs).sort_values(['subject','trials'])
    print(len(dfc0))

    me = dfc0.groupby(['subject','environment'])['err_sens'].\
        mean(numeric_only=1).reset_index()

    #sns.set(font_scale=1.3)
    fg = sns.catplot(data = me, kind='violin', y='err_sens', 
        x='environment')#,  order=['stable','random'])
    
    return dfc0
    # for ax in fg.axes.flatten():
    #     ax.set_ylim(-5,5)

In [ ]:
df_all_multi_tsz_3sept_3 = df_all_multi_tsz3
df_all_multi_tsz_3sept_2 = df_all_multi_tsz2
df_all_multi_tsz_28feb = pd.read_pickle('/home/demitau/data_Quentin/full_experiments/data2/df_all_multi_tsz.pkl')

In [ ]:
# df1 = truncateDf(df_all_multi_tsz_3sept, coln, q=0.0, infnan_handling='discard').sort_values(['subject','trials'])
# df2 = truncateDf(df_all_multi_tsz_28feb, coln, q=0.0, infnan_handling='discard').sort_values(['subject','trials'])

In [ ]:
addNonHitCol(df_all_multi_tsz_3sept), addNonHitCol(df_all_multi_tsz_28feb), 

In [ ]:
qs = 'trial_shift_size == 1 and trial_group_col_calc == "trialwe"'
df_all_multi_tsz_3sept_2_ = df_all_multi_tsz_3sept_2.query(qs)
df_all_multi_tsz_3sept_3_ = df_all_multi_tsz_3sept_3.query(qs)
df_all_multi_tsz_28feb_ = df_all_multi_tsz_28feb.query(qs)

In [ ]:
dfc0_28feb.columns

In [ ]:
dfc0_3sept.columns

In [ ]:
# df_all_multi_tsz_3sept_3inv = df_all_multi_tsz_3sept_3.copy()
# df_all_multi_tsz_3sept_3inv['err_sens'] = -df_all_multi_tsz_3sept_3inv['err_sens']

In [ ]:
dfc0_3sept_2 = plot_violins(df_all_multi_tsz_3sept_2)
dfc0_3sept_3 = plot_violins(df_all_multi_tsz_3sept_3)
#plot_violins(df_all_multi_tsz_3sept_3inv)
dfc0_28feb= plot_violins(df_all_multi_tsz_28feb)

In [ ]:
len(df_all_multi_tsz_3sept), len(df_all_multi_tsz_28feb)

In [ ]:
df_all_multi_tsz_28feb_.non_hit_shifted

In [ ]:
df_all_multi_tsz_3sept_.non_hit_shifted

In [ ]:
addNonHitCol(df_all)

In [ ]:
df_all.columns

In [ ]:
df_all[['trials','non_hit_not_adj']]

In [ ]:
with pd.option_context('display.min_rows',35,
                      'display.max_rows',35):
    display(
        df_all_multi_tsz_3sept_2_[['subject','trials','target_inds',
                                 'error','err_sens','non_hit_shifted','trial_group_col_calc']] )
    
    display(
            df_all_multi_tsz_3sept_3_[['subject','trials','target_inds',
                                 'error','err_sens','non_hit_shifted','trial_group_col_calc']] )
    
    
    display(
    df_all_multi_tsz_28feb_[['subject','trials','target_inds',
                                 'error','err_sens','non_hit_shifted','trial_group_col_calc']] )
        
        

In [ ]:
df3= df_all_multi_tsz_3sept_3_.set_index(['subject','trials'])
df2= df_all_multi_tsz_3sept_2_.set_index(['subject','trials'])
df0 = df_all_multi_tsz_28feb_.set_index(['subject','trials'])

In [ ]:
coln = 'err_sens'
r = df2[coln] -\
    df0[coln]
# wrong because joins nan and inf from both so no wonder r == 0
#rbad = (r == np.inf) | (r == -np.inf) | r.isna()
#r[~rbad].max()

In [ ]:
a1 = ((df3[coln] == np.inf) == (df0[coln] == np.inf))
a2 = ((df3[coln] == -np.inf) == (df0[coln] == -np.inf))
a3 = ((df3[coln].isna()) == (df0[coln].isna()))

bads = ((df3[coln] == np.inf) | (df3[coln] == -np.inf) | (df3[coln].isna()))
(a1 &a2 & a3).all()

In [ ]:
r = df3[coln] + df0[coln]
r[~bads].max()

In [ ]:
df0[coln]

In [ ]:
df3[coln]

In [ ]:
df3[['error','correction']]

In [ ]:
df0[['error','correction']]